<a href="https://colab.research.google.com/github/Archalharin/229352-stat-for-data-science/blob/main/660510632_Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,yes


### Data Exploration

In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum()) #จำนวนที่ข้อมูลเป็น Missing Values

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns: # ค่าที่เป็นไปได้แต่ละคอลลัม
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [5]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df["y"].map({'yes': 1, 'no':0 }) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x =df.drop(['y','y_new'], axis=1)

# Split the data BEFORE any transformations


# Print data shape
print(y)
print(x)


0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      n

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3 ) #test_size=70%

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
34461,31,entrepreneur,single,university.degree,no,no,no,cellular,may,thu,50,2,999,1,failure
25520,43,admin.,married,university.degree,no,no,no,cellular,nov,wed,97,1,999,0,nonexistent
35624,30,services,single,high.school,no,no,no,cellular,may,mon,111,2,999,0,nonexistent
32877,32,blue-collar,married,basic.9y,no,yes,yes,cellular,may,mon,93,2,999,0,nonexistent
36041,34,admin.,single,university.degree,no,no,no,cellular,may,tue,171,1,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12538,57,entrepreneur,married,university.degree,NaN,NaN,NaN,cellular,jul,mon,268,9,999,0,nonexistent
15784,43,admin.,married,high.school,no,yes,no,cellular,jul,mon,119,1,999,0,nonexistent
31040,53,admin.,married,university.degree,NaN,no,no,cellular,may,wed,160,7,999,1,failure
7119,43,blue-collar,married,basic.9y,no,no,no,telephone,may,thu,13,3,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [7]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo])

# Fit the scaler ONLY on the training data


# X_train_scaled_demo = # Write your code here
# X_test_scaled_demo = # Write your code here

In [8]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-1.897672e-16,257.480871,-5.101535e-17,962.838542,0.171274
std,1.000017e+00,257.281309,1.000017e+00,186.009985,0.490501
min,-2.204344e+00,0.000000,-5.722941e-01,0.000000,0.000000
25%,-7.644071e-01,102.000000,-5.722941e-01,999.000000,0.000000
50%,-1.884323e-01,179.000000,-2.085577e-01,999.000000,0.000000
75%,6.755300e-01,319.000000,1.551787e-01,999.000000,0.000000
max,5.571316e+00,4918.000000,1.470463e+01,999.000000,6.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [13]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(x_train_scaled_demo, columns=num_cols_demo, index=x_train.index).head())

print("\nMean of scaled 'age' (train):", x_train_scaled_demo[:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", x_train_scaled_demo[:, 1].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
28142 -1.532373 -0.572294
13256  0.099555 -0.572294
1613  -0.668411  0.155179
5503   0.291547 -0.572294
24100 -0.764407 -0.572294

Scaled X_train 'age' and 'campaign' head:


NameError: name 'x_train_scaled_demo' is not defined

**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [14]:
x_train['education']

,education
28142,university.degree
13256,basic.4y
1613,high.school
5503,high.school
24100,professional.course
...,...
23943,basic.4y
12174,basic.4y
28039,professional.course
30562,professional.course


In [15]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here

x_train[ord_col_demo]=imputer_ord.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo]=imputer_ord.transform(x_test[ord_col_demo])

x_train['education']

,education
28142,university.degree
13256,basic.4y
1613,high.school
5503,high.school
24100,professional.course
...,...
23943,basic.4y
12174,basic.4y
28039,professional.course
30562,professional.course


- **Ordinal Encoding**

In [16]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [17]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
x_train[ord_col_demo]= ordinal_encoder.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo]= ordinal_encoder.transform(x_test[ord_col_demo])

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
34461,-0.860403,entrepreneur,single,6.0,no,no,no,cellular,may,thu,50,-0.208558,999,1,failure
25520,0.291547,admin.,married,6.0,no,no,no,cellular,nov,wed,97,-0.572294,999,0,nonexistent
35624,-0.956399,services,single,4.0,no,no,no,cellular,may,mon,111,-0.208558,999,0,nonexistent
32877,-0.764407,blue-collar,married,3.0,no,yes,yes,cellular,may,mon,93,-0.208558,999,0,nonexistent
36041,-0.572415,admin.,single,6.0,no,no,no,cellular,may,tue,171,-0.572294,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12538,1.635488,entrepreneur,married,6.0,NaN,NaN,NaN,cellular,jul,mon,268,2.337597,999,0,nonexistent
15784,0.291547,admin.,married,4.0,no,yes,no,cellular,jul,mon,119,-0.572294,999,0,nonexistent
31040,1.251505,admin.,married,6.0,NaN,no,no,cellular,may,wed,160,1.610124,999,1,failure
7119,0.291547,blue-collar,married,3.0,no,no,no,telephone,may,thu,13,0.155179,999,0,nonexistent


Let's take a look at the imputed and ordinal-encoded `education`.

In [18]:
print("\nOriginal X_train 'education' head:")
print(x_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x.train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])


Original X_train 'education' head:
       education
29523        4.0
28745        1.0
27162        6.0
26118        3.0
29713        4.0

Imputed X_train 'education' head (after imputer.transform):


AttributeError: 'DataFrame' object has no attribute 'train'

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [19]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

x_train[nom_col_demo] = imputer_nom.transform(x_train[nom_col_demo])
x_test[nom_col_demo] = imputer_nom.transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [20]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here

x_train_onehot = onehot_encoder.fit_transform(x_train[['job']])
x_test_onehot = onehot_encoder.transform(x_test[['job']])

x_train_onehot.shape

(28831, 11)

In [21]:
x_train_onehot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [22]:
x_train = pd.concat([x_train.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
x_train = x_train.drop(['job'], axis=1)

x_train

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-1.532373,single,6.0,no,yes,no,telephone,apr,mon,86,-0.572294,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.099555,married,1.0,no,yes,no,telephone,jul,wed,52,-0.572294,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.668411,married,4.0,NaN,no,no,telephone,may,fri,483,0.155179,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.291547,married,4.0,no,yes,no,telephone,may,mon,312,-0.572294,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.764407,married,5.0,no,yes,no,telephone,nov,tue,486,-0.572294,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,1.827480,married,1.0,no,yes,no,cellular,aug,fri,94,-0.572294,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28827,1.443496,married,1.0,no,yes,no,telephone,jul,tue,92,-0.572294,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,-1.148390,single,5.0,no,no,no,cellular,apr,thu,444,-0.572294,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28829,1.251505,married,5.0,no,yes,no,telephone,may,mon,25,-0.572294,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
x_test = pd.concat([x_test.reset_index(drop=True),
                     pd.DataFrame(x_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
x_test = x_test.drop(['job'], axis=1)

x_test

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.860403,single,6.0,no,no,no,cellular,may,thu,50,-0.208558,999,1,failure,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.291547,married,6.0,no,no,no,cellular,nov,wed,97,-0.572294,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.956399,single,4.0,no,no,no,cellular,may,mon,111,-0.208558,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.764407,married,3.0,no,yes,yes,cellular,may,mon,93,-0.208558,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.572415,single,6.0,no,no,no,cellular,may,tue,171,-0.572294,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,1.635488,married,6.0,NaN,NaN,NaN,cellular,jul,mon,268,2.337597,999,0,nonexistent,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,0.291547,married,4.0,no,yes,no,cellular,jul,mon,119,-0.572294,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354,1.251505,married,6.0,NaN,no,no,cellular,may,wed,160,1.610124,999,1,failure,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355,0.291547,married,3.0,no,no,no,telephone,may,thu,13,0.155179,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("\nOriginal X_train 'job' head:")
print(X_train[nom_col_demo].iloc[40:45])
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=X_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", X_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(X_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=X_train.index).iloc[40:45])

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [ ]:
df1= pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df1

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [27]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
nominal_fe = ['job','marital','contact','poutcome']
ordinal_fe = ['education']
numerical_f = ['age','duration','campaign','pdays','previous']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

x_train_copy = x_train.copy()
x_test_copy = x_test.copy()


In [29]:
# 3.1 Mode Imputation for nominal variables
imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train_copy[nominal_features])

x_train_copy[nominal_features] = imputer_nom.fit_transform(x_train_copy[nominal_features])
x_test_copy[nominal_features] = imputer_nom.transform(x_test_copy[nominal_features])

In [31]:
# 3.1.2 One-Hot encoding for nominal variables
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_copy_onehot = onehot_encoder.fit_transform(x_train_copy[nominal_features])
x_test_copy_onehot = onehot_encoder.transform(x_test_copy[nominal_features])

x_train_copy = pd.concat([x_train_copy.reset_index(drop=True),
                     pd.DataFrame(x_train_copy_onehot, columns=onehot_encoder.get_feature_names_out(nominal_features))], axis=1)
x_train_copy = x_train_copy.drop(nominal_features, axis=1)

x_test_copy = pd.concat([x_test_copy.reset_index(drop=True),
                     pd.DataFrame(x_test_copy_onehot, columns=onehot_encoder.get_feature_names_out(nominal_features))], axis=1)
x_test_copy = x_test_copy.drop(nominal_features, axis=1)

In [35]:
# 3.2.1 Mode Imputation for ordinal variables
imputer_ord = SimpleImputer(strategy='most_frequent')

x_train_copy[ord_col_demo] = imputer_ord.fit_transform(x_train_copy[ord_col_demo])
x_test_copy[ord_col_demo] = imputer_ord.transform(x_test_copy[ord_col_demo])

In [36]:
# 3.2.2 Ordinal encoding for ordinal variables
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

ordinal_encoder = OrdinalEncoder(categories=[education_categories]) # ordinal ของ sci-kit learn

x_train_copy[ord_col_demo] = ordinal_encoder.fit_transform(x_train_copy[ord_col_demo])
x_test_copy[ord_col_demo] = ordinal_encoder.transform(x_test_copy[ord_col_demo])

In [37]:
# 3.3 Scaler for numerical variables
scaler = StandardScaler()

x_train_copy[numerical_features] = scaler.fit_transform(x_train_copy[numerical_features])
x_test_copy[numerical_features] = scaler.transform(x_test_copy[numerical_features])

In [38]:
# 3.4 Variables selection
x_train_copy = x_train_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)
x_test_copy = x_test_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)

In [39]:
# 4.Train Logistic Regression
model = LogisticRegression()
model.fit(x_train_copy, y_train)

y_pred = model.predict(x_test_copy)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [42]:

target_names = ['non-subscribed a term deposit', 'subscribed a term deposit']
print(classification_report(y_test, y_pred, target_names=target_names))

                               precision    recall  f1-score   support

non-subscribed a term deposit       0.92      0.98      0.95     10940
    subscribed a term deposit       0.68      0.35      0.46      1417

                     accuracy                           0.91     12357
                    macro avg       0.80      0.66      0.71     12357
                 weighted avg       0.89      0.91      0.89     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

Number of training documents: 11314
Number of test documents: 7532
Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse =vectorizer.fit_transform(X_train_news) # Write your code here
test_output = vectorizer.transform(X_test_news)

sample_vec_output_dense = sample_vec_output_sparse.toarray()

# print(vectorizer.vocabulary_)
# print(vectorizer.get_feature_names_out())
# print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


